This notebook generates the paragraph about the microCT-scanning from logfiles of the scans.

In [1]:
import platform
import os
import pandas
import glob
import numpy

We select a folder and go through *each* subfolder there...

In [2]:
# Different locations if running either on Linux or Windows
if 'Linux' in platform.system():
    BasePath = os.path.join(os.sep, 'home', 'habi', 'research-storage-uct', 'Archiv_Tape')
else:
    BasePath = os.path.join('R:\\', 'Archiv_Tape')
Root = os.path.join(BasePath, '*Carolina*Muscles*')
print('We are loading all the data from %s' % Root)

We are loading all the data from /home/habi/research-storage-uct/Archiv_Tape/*Carolina*Muscles*


In [3]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [4]:
# Look for *all* log files in the selected folder
Data['LogFile'] = sorted(glob.glob(os.path.join(Root, '**', '*.log'), recursive=True))

In [5]:
# Use only the 'proj' log files
for c, row in Data.iterrows():
    if 'rec' not in row.LogFile:
        Data.drop([c], inplace=True)
    if 'rectmp' in row.LogFile:
        Data.drop([c], inplace=True)
    if 'ctan.log' in row.LogFile:
        # Remove log file from CTAn
        Data.drop([c], inplace=True)        
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [6]:
# Show what we have
for l in Data.LogFile:
    print(l)

/home/habi/research-storage-uct/Archiv_Tape/Zebrafish_Carolina_Muscles/ko01/rec/ko01_rec.log
/home/habi/research-storage-uct/Archiv_Tape/Zebrafish_Carolina_Muscles/ko02/rec/ko02_rec.log
/home/habi/research-storage-uct/Archiv_Tape/Zebrafish_Carolina_Muscles/ko03/rec/ko03_rec.log
/home/habi/research-storage-uct/Archiv_Tape/Zebrafish_Carolina_Muscles/ko04/rec/ko04_rec.log
/home/habi/research-storage-uct/Archiv_Tape/Zebrafish_Carolina_Muscles/ko05/rec/ko05_rec.log
/home/habi/research-storage-uct/Archiv_Tape/Zebrafish_Carolina_Muscles/wt01/rec/wt01_rec.log
/home/habi/research-storage-uct/Archiv_Tape/Zebrafish_Carolina_Muscles/wt02/rec/wt02_rec.log
/home/habi/research-storage-uct/Archiv_Tape/Zebrafish_Carolina_Muscles/wt03/rec/wt03_rec.log
/home/habi/research-storage-uct/Archiv_Tape/Zebrafish_Carolina_Muscles/wt04/rec/wt04_rec.log
/home/habi/research-storage-uct/Archiv_Tape/Zebrafish_Carolina_Muscles/wt05/rec/wt05_rec.log


In [7]:
# Get 'scan' folder
Data['Scan'] = [os.path.basename(os.path.dirname(os.path.dirname(l))) for l in Data['LogFile']]

Define us a bunch of functions to `grep` the necessary things from the log files

In [8]:
def fulllog(logfile):
    with open(logfile, 'r') as f:
        for line in f:
            print(line.strip())
    return()

In [9]:
# Temporarily select one random log file for testing the functions
log = Data['LogFile'][Data.sample().index[0]]

In [10]:
# Print us the full log file
# fulllog(log)

In [11]:
import re
def scanner(logfile, verbose=False):
    hardwareversion = []
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scanner' in line:
                if verbose:
                    print(line)
                # Sometimes it's SkyScan, sometimes Skyscan, so we have to regex it :)
                machine = re.split('Sky.can', line)[1].strip()
            if 'Hardware' in line:
                if verbose:
                    print(line)
                hardwareversion = line.split('=')[1].strip()
    if hardwareversion:
        return('SkyScan %s (Version %s)' % (machine, hardwareversion))
    else:
        return('SkyScan ' + machine)    

In [12]:
scanner(log, verbose=True)

Scanner=SkyScan1272



'SkyScan 1272'

In [13]:
def controlsoftware(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Software Ver' in line:
                if verbose:
                    print(line)
                version = line.split('=')[1].strip()
    return(version)

In [14]:
controlsoftware(log, verbose=True)

Software Version=1.1.19



'1.1.19'

In [15]:
def source(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Source Ty' in line:
                if verbose:
                    print(line)
                source = line.split('=')[1].strip()
                if 'HAMAMA' in source:
                    # We split the string at '_L' to separate HAMAMATSU_L118...
                    # Afterwards we properly capitalize HAMAMATSU and
                    # join the strings back with ' L' to get the beginning of the reference back
                    source = ' L'.join([s.capitalize() for s in source.split('_L')])
    return(source)

In [16]:
source(log, verbose=True)

Source Type=HAMAMATSU_L11871_20



'Hamamatsu L11871_20'

In [17]:
def camera(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Camera T' in line or 'Camera=' in line:
                if verbose:
                    print(line)
                cam = line.split('=')[1].strip().strip(' camera')
    return(cam)

In [18]:
camera(log, verbose=True)

Camera Type=XIMEA xiRAY16



'XIMEA xiRAY16'

In [19]:
def voltage(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Voltage' in line:
                if verbose:
                    print(line)
                V = float(line.split('=')[1])
    return(V)

In [20]:
voltage(log, verbose=True)

Source Voltage (kV)=  70



70.0

In [21]:
def current(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Source Current' in line:
                if verbose:
                    print(line)
                A = float(line.split('=')[1])
    return(A)

In [22]:
current(log, verbose=True)

Source Current (uA)= 142



142.0

In [23]:
def whichfilter(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Filter=' in line:
                if verbose:
                    print(line)
                fltr = line.split('=')[1].strip().replace('  ', ' ')
                if fltr=='No Filter':
                    fltr=False
    return(fltr)

In [24]:
whichfilter(log, verbose=True)

Filter=Al 0.5mm



'Al 0.5mm'

In [25]:
def numproj(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'f Files' in line:
                if verbose:
                    print(line)
                numproj = int(line.split('=')[1])
    return(numproj)

In [26]:
numproj(log, verbose=True)

Number Of Files=  482



482

In [27]:
def stacks(logfile, verbose=False):
    with open(logfile, 'r') as f:
        # If only one stack, then there's nothing in the log file
        numstacks = 0
        for line in f:
            if 'of conn' in line:
                if verbose:
                    print(line)
                # The 'Sub-scan scan length' is listed in the log file
                # We simply select the last one, and add 1, since Bruker also starts to count at zero
                numstacks = int(line.split('=')[1])
    return(numstacks)

In [28]:
stacks(log, verbose=True)

Number of connected scans=5



5

In [29]:
def camerasize(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Columns' in line:
                if verbose:
                    print(line)
                columns = int(line.split('=')[1])
            if 'Rows' in line:
                if verbose:
                    print(line)
                rows = int(line.split('=')[1])
    return(columns, rows)

In [30]:
camerasize(log, verbose=True)

Number Of Rows= 1092

Number Of Columns= 1632



(1632, 1092)

In [31]:
def overlapscan(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Horizontal Off' in line:
                if verbose:
                    print(line)
                wide = int(line.split('=')[1])
                if wide == 1:
                    wide=False
    return(wide)

In [32]:
overlapscan(log, verbose=True)

Number Of Horizontal Offset Positions=1



False

In [33]:
def threesixtyscan(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if '0 Rotation' in line:
                if verbose:
                    print(line)
                threesixty = line.split('=')[1].strip()
                if threesixty == 'YES':
                    threesixty = True
                else:
                    threesixty = False
    return(threesixty)

In [34]:
threesixtyscan(log, verbose=True)

Use 360 Rotation=NO



False

In [35]:
def rotationstep(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Rotation Step' in line:
                if verbose:
                    print(line)
                rotstep = float(line.split('=')[1])
    return(rotstep)

In [36]:
rotationstep(log, verbose=True)

Rotation Step (deg)=0.400



0.4

In [37]:
def exposure(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Exposure' in line:
                if verbose:
                    print(line)
                exp = int(line.split('=')[1])
    return(exp)

In [38]:
exposure(log, verbose=True)

Exposure (ms)=862



862

In [39]:
def averaging(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Avera' in line:
                if verbose:
                    print(line)
                details = line.split('=')[1]
                if 'ON' in details:
                    # https://stackoverflow.com/a/4894156/323100
                    avg = int(details[details.find("(")+1:details.find(")")])
                else:
                    avg=False
    return(avg)

In [40]:
averaging(log, verbose=True)

Frame Averaging=ON (3)



3

In [41]:
import datetime
def duration(logfile, verbose=False):
    '''Returns scantime in *seconds*'''
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scan duration' in line:
                if verbose:
                    print(line)
                duration = line.split('=')[1].strip()
    # Sometimes it's '00:24:26', sometimes '0h:52m:53s' :-/
    if 'h' in duration:
        scantime = datetime.datetime.strptime(duration, '%Hh:%Mm:%Ss')
    else:
        scantime = datetime.datetime.strptime(duration, '%H:%M:%S')
    return((scantime-datetime.datetime(1900,1,1)).total_seconds())

In [42]:
duration(log, verbose=True)

Scan duration=0h:36m:49s



2209.0

In [43]:
def timeformat(tdelta, fmt):
    # From https://stackoverflow.com/a/8907269/323100
    d = {"days": tdelta.days}
    d["hours"], rem = divmod(tdelta.seconds, 3600)
    d["minutes"], d["seconds"] = divmod(rem, 60)
    return fmt.format(**d)

In [44]:
print('Each stack took', end=' ')
if duration(log)/3600 > 1:
    # Scan took hours
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
else:
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{minutes} minutes'), end=' ')
print('to scan')

Each stack took 36 minutes to scan


In [45]:
def pixelsize(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Image Pixel' in line and 'Scaled' not in line:
                if verbose:
                    print(line)
                pixelsize = float(line.split('=')[1])
    return(pixelsize)

In [46]:
pixelsize(log, verbose=True)

Image Pixel Size (um)=7.999950



7.99995

In [47]:
def version(logfile, verbose=False):
    Program = numpy.nan
    Version = numpy.nan
    with open(logfile, 'r') as f:
        for line in f:
            if 'Reconstruction Program' in line:
                if verbose:
                    print(line)
                Program = line.split('=')[1].strip()
            if 'Program Version' in line:
                if verbose:
                    print(line)
                Version = line.split('sion:')[1].strip()
    return(Program, Version)

In [48]:
version(log, verbose=True)

Reconstruction Program=NRecon

Program Version=Version: 2.0.0.5



('NRecon', '2.0.0.5')

In [49]:
def ringremoval(logfile, verbose=False):
    ring = numpy.nan
    with open(logfile, 'r') as f:
        for line in f:
            if 'Ring' in line:
                if verbose:
                    print(line)
                ring = int(line.split('=')[1].strip())
    return(ring)

In [50]:
ringremoval(log, verbose=True)

Ring Artifact Correction=7



7

In [51]:
def beamhardening(logfile, verbose=False):
    bh = numpy.nan
    with open(logfile, 'r') as f:
        for line in f:
            if 'ardeni' in line:
                if verbose:
                    print(line)
                bh = int(line.split('=')[1].strip())
    return(bh)

In [52]:
beamhardening(log, verbose=True)

Beam Hardening Correction (%)=0



0

In [53]:
def get_reconstruction_grayvalue(logfile, verbose=False):
    grayvalue = None
    """How did we map the brightness of the reconstructions?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Maximum for' in line:
                if verbose:
                    print(line)
                grayvalue = float(line.split('=')[1])
    return(grayvalue)

In [54]:
get_reconstruction_grayvalue(log, verbose=True)

Maximum for CS to Image Conversion=0.150000



0.15

Extract *all* values for *all* log files.

In [55]:
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Software'] = [controlsoftware(log) for log in Data['LogFile']]

In [56]:
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]
Data['Voxelsize_rounded'] = [round(vs,1) for vs in Data['Voxelsize']]

In [57]:
Data['Source'] = [source(log) for log in Data['LogFile']]
Data['Camera'] = [camera(log) for log in Data['LogFile']]

In [58]:
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]

In [59]:
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['NumProj'] = [numproj(log) for log in Data['LogFile']]
Data['CamSize'] = [camerasize(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Wide'] = [overlapscan(log) for log in Data.LogFile]
Data['ThreeSixty'] = [threesixtyscan(log) for log in Data['LogFile']]

In [60]:
Data['RingRemoval'] = [ringremoval(log) for log in Data['LogFile']]
Data['Beamhardening'] = [beamhardening(log) for log in Data['LogFile']]
Data['GrayValueMax'] = [get_reconstruction_grayvalue(log) for log in Data['LogFile']]

In [61]:
Data['Exposure'] = [exposure(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]

In [62]:
Data['Duration'] = [duration(log) for log in Data['LogFile']]

In [63]:
Data['Version'] = [version(log) for log in Data['LogFile']]

In [64]:
# Save out data to CSV, so we can link to it in the manuscript.
Data.to_csv(os.path.join('data','cox7a-tomography.csv'))

In [65]:
Data

,LogFile,Scan,Scanner,Software,Voxelsize,Voxelsize_rounded,Source,Camera,Voltage,Current,...,RotationStep,Wide,ThreeSixty,RingRemoval,Beamhardening,GrayValueMax,Exposure,Averaging,Duration,Version
0,/home/habi/research-storage-uct/Archiv_Tape/Ze...,ko01,SkyScan 1272,1.1.19,7.99995,8.0,Hamamatsu L11871_20,XIMEA xiRAY16,70.0,142.0,...,0.4,False,False,7,0,0.15,862,3,2206.0,"(NRecon, 2.0.0.5)"
1,/home/habi/research-storage-uct/Archiv_Tape/Ze...,ko02,SkyScan 1272,1.1.19,7.99995,8.0,Hamamatsu L11871_20,XIMEA xiRAY16,70.0,142.0,...,0.4,False,False,7,0,0.15,862,3,2209.0,"(NRecon, 2.0.0.5)"
2,/home/habi/research-storage-uct/Archiv_Tape/Ze...,ko03,SkyScan 1272,1.1.19,7.99995,8.0,Hamamatsu L11871_20,XIMEA xiRAY16,70.0,142.0,...,0.4,False,False,7,0,0.15,862,3,2208.0,"(NRecon, 2.0.0.5)"
3,/home/habi/research-storage-uct/Archiv_Tape/Ze...,ko04,SkyScan 1272,1.1.19,7.99995,8.0,Hamamatsu L11871_20,XIMEA xiRAY16,70.0,142.0,...,0.4,False,False,7,0,0.15,862,3,2207.0,"(NRecon, 2.0.0.5)"
4,/home/habi/research-storage-uct/Archiv_Tape/Ze...,ko05,SkyScan 1272,1.1.19,7.99995,8.0,Hamamatsu L11871_20,XIMEA xiRAY16,70.0,142.0,...,0.4,False,False,7,0,0.15,862,3,2206.0,"(NRecon, 2.0.0.5)"
5,/home/habi/research-storage-uct/Archiv_Tape/Ze...,wt01,SkyScan 1272,1.1.19,7.99995,8.0,Hamamatsu L11871_20,XIMEA xiRAY16,70.0,142.0,...,0.4,False,False,7,0,0.15,862,3,2209.0,"(NRecon, 2.0.0.5)"
6,/home/habi/research-storage-uct/Archiv_Tape/Ze...,wt02,SkyScan 1272,1.1.19,7.99995,8.0,Hamamatsu L11871_20,XIMEA xiRAY16,70.0,142.0,...,0.4,False,False,7,0,0.15,862,3,2210.0,"(NRecon, 2.0.0.5)"
7,/home/habi/research-storage-uct/Archiv_Tape/Ze...,wt03,SkyScan 1272,1.1.19,7.99995,8.0,Hamamatsu L11871_20,XIMEA xiRAY16,70.0,142.0,...,0.4,False,False,7,0,0.15,862,3,2206.0,"(NRecon, 2.0.0.5)"
8,/home/habi/research-storage-uct/Archiv_Tape/Ze...,wt04,SkyScan 1272,1.1.19,7.99995,8.0,Hamamatsu L11871_20,XIMEA xiRAY16,70.0,142.0,...,0.4,False,False,7,0,0.15,862,3,2209.0,"(NRecon, 2.0.0.5)"
9,/home/habi/research-storage-uct/Archiv_Tape/Ze...,wt05,SkyScan 1272,1.1.19,7.99995,8.0,Hamamatsu L11871_20,XIMEA xiRAY16,70.0,142.0,...,0.4,False,False,7,0,0.15,862,3,2213.0,"(NRecon, 2.0.0.5)"


----

Pour all the values into text, according to my standard microct blurb from http://simp.ly/publish/NBhZhH

In [66]:
print('Based on the %s log files in %s' % (len(Data), Root))

Based on the 10 log files in /home/habi/research-storage-uct/Archiv_Tape/*Carolina*Muscles*


In [67]:
print('After $PREPARATION, the',
      len(Data),
      'samples were imaged on a Bruker',
      " OR ".join(str(value) for value in Data.Scanner.unique()),
      'high-resolution microtomography machine (Control software version',
      " OR ".join(str(value) for value in Data.Software.unique()) + 
      ', Bruker microCT, Kontich, Belgium).')

After $PREPARATION, the 10 samples were imaged on a Bruker SkyScan 1272 high-resolution microtomography machine (Control software version 1.1.19, Bruker microCT, Kontich, Belgium).


In [68]:
print('The machine is equipped with a',
      " OR ".join(str(value) for value in Data.Source.unique()),
      'X-ray source and a',
      " OR ".join(str(value) for value in Data.Camera.unique()),
      'camera.')

The machine is equipped with a Hamamatsu L11871_20 X-ray source and a XIMEA xiRAY16 camera.


In [69]:
print('The X-ray source was set to a tube voltage of', 
      " OR ".join(str(value) for value in Data.Voltage.unique()),
      'kV and a tube current of',
      " OR ".join(str(value) for value in Data.Current.unique()),
      'µA, the x-ray spectrum was', end=' ')
if Data.Filter.unique():
    print('filtered by', " OR ".join(str(value) for value in Data.Filter.unique()), end=' ')
else:
    print('not filtered', end=' ')
print('prior to incidence onto the sample.')

The X-ray source was set to a tube voltage of 70.0 kV and a tube current of 142.0 µA, the x-ray spectrum was filtered by Al 0.5mm prior to incidence onto the sample.


In [70]:
Data.ThreeSixty.unique()

array([False])

In [71]:
print('For each sample, we recorded a set of', end=' ')
if Data.Filter.unique().tolist():   
    print(" or ".join(str(value) for value in Data.Stacks.unique()),
          'stacked scans overlapping the sample height, each stack was recorded with', end=' ')
print(" or ".join(str(value) for value in Data.NumProj.unique()), 'projections of', end=' ')
for cs in Data.CamSize.unique():
    print(cs[0], end=' ')
print('x', end=' ')
for cs in Data.CamSize.unique():
    print(cs[1], end=' ')
print('pixels', end=' ')
if Data.Wide.unique():
    print('(' + " or ".join(str(value) for value in Data.Wide.unique()), 'projections stitched laterally)', end=' ')
print('at every',
       str(" or ".join(str(value) for value in Data.RotationStep.unique())) + '° over a ', end='')
if Data.ThreeSixty.unique():
     print('360°', end=' ')
else:
    print('180°', end=' ')
print('sample rotation.')

For each sample, we recorded a set of 5 stacked scans overlapping the sample height, each stack was recorded with 482 projections of 1632 x 1092 pixels at every 0.4° over a 180° sample rotation.


In [72]:
print('Every single projection was exposed for',
      " or ".join(str(value) for value in Data.Exposure.unique()),
      'ms,',
      " or ".join(str(value) for value in Data.Averaging.unique()),
      'projections were averaged to greatly reduce image noise.')

Every single projection was exposed for 862 ms, 3 projections were averaged to greatly reduce image noise.


In [73]:
print('This resulted in a scan time of approximately ', end='')
if duration(log)/3600 > 1:
    # Scan took hours
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
else:
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{minutes} minutes'), end=' ')
if not stacks(log) == 1:
    print('per stack and about',
          timeformat(stacks(log) * datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
print('per sample', end='')
if stacks(log) == 1:
    print('.')
else:
    print(' (with', stacks(log), 'stacks each).')

This resulted in a scan time of approximately 36 minutes per stack and about 3 hours and 4 minutes per sample (with 5 stacks each).


In [74]:
print('In total, we scanned', Data.Stacks.sum(), 'stacks.')
print('Each stack took approximately',
      Data.Duration.mean() // 60,
      'minutes (' + str(datetime.timedelta(seconds=Data.Duration.mean())) + ')')
print('In total, we thus scanned for about', 
      timeformat(Data.Stacks.sum() *
                 datetime.timedelta(seconds=Data.Duration.mean()),
                 '{days} days, {hours} hours and {minutes} minutes.'))
hourlyrate = 125
print('At the MIC rate of %s CHF/h, this would have cost %s CHF' % (
    hourlyrate,
    int(round(Data.Stacks.sum() * Data.Duration.mean() / 60 / 60 * hourlyrate))))

In total, we scanned 50 stacks.
Each stack took approximately 36.0 minutes (0:36:48.300000)
In total, we thus scanned for about 1 days, 6 hours and 40 minutes.
At the MIC rate of 125 CHF/h, this would have cost 3834 CHF


In [75]:
print('The projection images were then subsequently reconstructed into a 3D stack',
      'of images with',
      Data.Version.unique()[0][0],
      '(Version',
      version(log)[1] + ', Bruker microCT, Kontich Belgium)', end=' ')
if ringremoval(log):
    print('using a ring artifact correction of',
          ringremoval(log), end='')
if beamhardening(log):
    print(' and a beam hardening correction of',
          beamhardening(log),
          '%.')
else:
    print('.')

The projection images were then subsequently reconstructed into a 3D stack of images with NRecon (Version 2.0.0.5, Bruker microCT, Kontich Belgium) using a ring artifact correction of 7.


In [76]:
print('The whole process resulted in datasets with an isometric voxel size of',
      " or ".join(str(value) for value in Data.Voxelsize_rounded.unique()),
      'µm.') 

The whole process resulted in datasets with an isometric voxel size of 8.0 µm.
